# Bulk-transcriptomics Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import gseapy as gp
from matplotlib.patches import Patch
import mygene
import matplotlib.patheffects as PathEffects
from adjustText import adjust_text
import dask.dataframe as dd


##### In the first step, the expected counts from each of the individual RSEM output files are combined together to create the combined count matrix for the whole dataset

In [ ]:

def create_combined_count_matrix(folder_path):
    """
    Create a combined count matrix from all the individual RSEM files in the specified folder.

    Parameters:
        - folder_path (str): Path to the folder containing the RSEM files.
    Returns:
        - count_matrix (pandas DataFrame): The combined count matrix for the dataset
    """
    # empty list to store the individual count dataframes
    count_dfs = []

    # Iterate over all RSEM output files in the folder
    for file_name in os.listdir(folder_path):
       
        # Generate the full path to access file
        file_path = os.path.join(folder_path, file_name)
        
        # Read the file into a Dask DataFrame, extract the gene_id and expected_count columns
        count_df = dd.read_csv(file_path, sep='\t', usecols=['gene_id', 'expected_count'])
        
        # Assign the sample name (derived from file name) to the expected_count columns
        sample_name = file_name.split(".")[0]
        count_df = count_df.rename(columns={'expected_count': sample_name})
            
        # change the index to gene_id column
        count_df = count_df.set_index('gene_id')
            
        # Add the count data frame to the count_dfs list
        count_dfs.append(count_df)

    # Merge all individual count data frames based on same index (gene_id)
    count_matrix = dd.concat(count_dfs, axis=1)
    # Fill NaN values with 0 (in case of missing values)
    count_matrix = count_matrix.fillna(0)
    # Round off the expected counts to nearest integer values which is necessary for DESeq2
    count_matrix = count_matrix.round().astype(int)
    # Compute the final DataFrame
    count_matrix = count_matrix.compute()
    #return the count matrix
    return count_matrix

# The folder path contating the RSEM files
folder_path = '../Bulk-Transcriptomics/GSE205099_RAW/'
# Generate the combined count matrix
counts_df = create_combined_count_matrix(folder_path)
# Save the count matrix
counts_df.to_csv(f'../Bulk-Transcriptomics/combined_count_matrix.tsv', sep='\t')
print("Combined Count matrix created successfully!")


##### We convert the Ensembl ids to corresponding gene symbol. In case of multiple Ensembl IDs pointing to the same genesymbol, we added their counts together for the respective replicates

In [ ]:

def process_counts_df(counts_df):
    """
    To Process the counts dataframe by mapping the Ensembl IDs to Corresponding Gene Symbols and Perform additional transformations.

    Parameters:
        - counts_df (pandas DataFrame): The counts matrix dataframe containing Ensembl IDs as index and gene counts as columns.
    Returns:
        - counts_df (pandas DataFrame): The processed counts matrix dataframe.
    """
    # Initialize the mygene object
    my_gene_object = mygene.MyGeneInfo()
    
    # Extract the Ensembl IDs without version numbers(if present) 
    ensembl_ids = counts_df.index.str.split('.').str[0].unique()

    # Query the IDs to fetch the corresponding Gene Symbols
    query_results = my_gene_object.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='human')

    # Using the Query result a dictionary is created to map Ensembl IDs to corresponding Gene Symbols
    ensembl_to_symbol = {item['query']: item.get('symbol', 'NaN') for item in query_results}

    # Add the correspodning Gene Symbols to the counts dataframe
    counts_df['GeneSymbol'] = counts_df.index.str.split('.').str[0].map(ensembl_to_symbol)

    # Change the index of the count matrix dataframe to the GeneSymbol column
    counts_df.set_index('GeneSymbol', inplace=True)

    # Transpose the dataframe
    counts_df = counts_df.T

    # Merge counts of the same but repeated gene symbols
    counts_df = counts_df.groupby(level=0, axis=1).sum()
    
    # Return the processed
    return counts_df


counts_df=process_counts_df(counts_df)

##### Next, we Create the metadata dataframe highlighting the biological condition of each replicate which is required for carrying out the Differential Expression Analysis

In [ ]:
def assign_condition(replicate):
    """
    Determines the condition of a given replicate based on its name.

    Parameters:
        - replicate (str): The name of the replicate.
    Returns:
        - str: The condition of the replicate. Either "COVID-19" or "Control".
    """
    
    if replicate.split("_")[1] == "covid19":
        return "COVID-19"
    else:
        return "Control"

# Extract the replicate name as a list
replicates=list(counts_df.index)
# Generate a list of the assocaited biological condition of each replicate     
condition_list=[assign_condition(sample) for sample in replicates]
# Generate the metadata 
metadata=pd.DataFrame(condition_list, index=replicates, columns=["Condition"])

metadata

##### We now carry out the Differential Expression Analysis using DESeq2 by Comparing Infected Samples against Control 

In [ ]:
# Filter out genes with low counts
counts_df = counts_df.loc[:, counts_df.sum() >= 10]

# Initialize the DESeq2 object
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata,
    design_factors="Condition",
    n_cpus=32,  # n_cpus can be specified here or in the inference object
)

dds.deseq2()

# Run the DESeq2 analysis, comparing the Covid against Control conditions
stat_res = DeseqStats(dds, contrast=('Condition', 'COVID-19', 'Control'))

# Summarize the results
stat_res.summary()

# Store the results in this dataframe
dea_results = stat_res.results_df


##### Next, we visulize these results. First, we print the DEA results and construct the Volcano plot higlighting the genes of interest

In [ ]:
# Genes of Interest
gene_list=["ZC3HAV1",
           'AICDA', 'APOBEC1', 'APOBEC2', 'APOBEC3A', 'APOBEC3B', 'APOBEC3C', 'APOBEC3D', 'APOBEC3F', 'APOBEC3G', 'APOBEC3H', 'APOBEC4',
           'ADAR', 'ADARB1', 'ADARB2']

In [ ]:
# make directory to store the bulk transcriptomic analysis results images
os.makedirs('../Results/bulk_transcriptomic_analysis/', exist_ok=True)

In [ ]:
# See the DESEQ2 results for the genes of interest
dea_results[dea_results.index.isin(gene_list)].to_csv(f'../Bulk-Transcriptomics/DESeq2_results_genes_of_interest.tsv')

In [ ]:


def plot_volcano(data, gene_list, logfc_threshold=.75, padj_threshold=0.05, adjust=True):
    """
    Plot a volcano plot to visualize differential gene expression analysis results.

    Parameters:
    - data (pandas.DataFrame): The data containing gene expression analysis results.
    - gene_list (list): A list of genes of interest to be highlighted in the plot.
    - logfc_threshold (float, optional): The threshold for log2 fold change. Default is 0.75.
    - padj_threshold (float, optional): The threshold for adjusted p-value. Default is 0.1.
    - adjust (bool, optional): Whether to adjust the position of gene labels to avoid overlap. Default is True.
    """
    plt.figure(figsize=(10, 10))

    # Filter points based on thresholds
    nonsig_points = data[(data['padj'] > padj_threshold) | (abs(data['log2FoldChange']) < logfc_threshold)]
    sig_points = data[(data['padj'] <= padj_threshold) & (abs(data['log2FoldChange']) >= logfc_threshold)]
    sig_pval_only = data[data['padj'] <= padj_threshold]

    # Plot non-significant points
    plt.scatter(nonsig_points['log2FoldChange'], -np.log10(nonsig_points['padj']), c='#DCDCDC', edgecolor='#E0E0E0', label='Not Statistically Significant', marker='o')

    # Plot significant points
    plt.scatter(sig_points['log2FoldChange'], -np.log10(sig_points['padj']), c='#A9A9A9', edgecolor='#E0E0E0', label='Statistically Significant', marker='o')

    # Plot genes in the filtered gene list with a different marker
    genes_of_interest = data[data.index.isin(gene_list)]
    plt.scatter(genes_of_interest['log2FoldChange'], -np.log10(genes_of_interest['padj']), c='b', edgecolor='none', label='Genes of Interest', marker='^')

    # Draw horizontal line for padj threshold
    plt.axhline(y=-np.log10(padj_threshold), color='#909090', linestyle='--', linewidth=1)

    # Draw vertical lines for log2 fold change thresholds
    plt.axvline(x=logfc_threshold, color='#909090', linestyle='--', linewidth=1)
    plt.axvline(x=-logfc_threshold, color='#909090', linestyle='--', linewidth=1)

    # Label points in the filtered gene list
    texts = []
    for gene in gene_list:
        gene_data = data[data.index == gene]
        if not gene_data.empty:
            x = gene_data['log2FoldChange'].values[0]
            y = -np.log10(gene_data['padj'].values[0])
            gene_label = gene + (r'$^{★}$' if gene in sig_pval_only.index else '')
            txt = plt.text(x, y, gene_label, size=12, ha='center', va='center')  
            txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
            texts.append(txt)
    
    # Add legend for the statistically significant genes of interest which are highlighted with a '★' marker
    plt.scatter([], [], c='black', marker="*",edgecolor='none', s=70, label=f'Statistically Significant Genes of \nInterest ($p_{{adj}}$ < {padj_threshold})')


    plt.legend(fontsize=12) 
    plt.xlabel("$log_{2}$ fold change", size=15)
    plt.ylabel("-$log_{10}$ $p_{adj}$-value", size=15)

    if adjust:
        adjust_text(texts, arrowprops=dict(arrowstyle="-", color='k', lw=0.5))
    
    # Save the plot
    plt.savefig('../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_volcano_plot.png', dpi = 1000)
    print("The Volcano plot has been saved at '../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_volcano_plot.png'")
    
    plt.show()


# Explicitly Add the GeneSymbol column to the DataFrame for plotting
dea_results["GeneSymbol"] = dea_results.index

# Call the function to plot the volcano plot highlighting the genes of interest
plot_volcano(dea_results, gene_list)

##### Next, we carry out Gense set enrichment for the Oxidative Damage Response geneset using GSEApy

In [ ]:

def run_gene_set_enrichment_analysis(dea_results, oxidative_stress_gene_set):
    """
    Runs gene set enrichment analysis using GSEApy for a specified gene set.

    Args:
        dea_results(pandas dataframe): DESeq2 results with columns 'GeneSymbol', 'padj', and 'log2FoldChange'.
        oxidative_stress_gene_set (str): Path to the Oxidative Damage response gene set file.
    """

    # Create a ranking of genes based on the DESeq2 results
    dea_results["-log_padj x lfc"] = -np.log(dea_results["padj"]) * dea_results["log2FoldChange"]
    # Sort the genes by the -log_padj x lfc
    ranked_genes = dea_results[['GeneSymbol', '-log_padj x lfc']].dropna().sort_values('-log_padj x lfc', ascending=False)
    # Reset the index to GeneSymbol column
    ranked_genes.reset_index(drop=True, inplace=True)

    # Run Gene Set Enrichment Analysis
    pre_res = gp.prerank(rnk=ranked_genes, gene_sets=oxidative_stress_gene_set, seed=6, min_size=0, max_size=200)

    # Plot the results
    pre_res.plot(terms="WP_OXIDATIVE_DAMAGE_RESPONSE", figsize=(10, 10))
    
    plt.title("")
    
    # Save the figure
    plt.savefig("../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_gsea_plot.png", dpi=1000)
    print("The GSEA plot has been saved at '../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_gsea_plot.png'")
    
    plt.show()   

run_gene_set_enrichment_analysis(dea_results, "../WP_OXIDATIVE_DAMAGE_RESPONSE.gmt")

##### In order to invesgate the variability of the genes of interest among different replicates we plot the clustermap

In [ ]:

def plot_normalized_expression_clustermap(dds, gene_list, metadata):
    """
    Plot a heatmap using the genes of interest for the different replicates.

    Args:
        dds (object): The DESeq2 object containing normalized counts.
        gene_list (list): The list of genes to be used in constructing the heatmap.
        metadata (DataFrame): The DataFrame containing metadata information of the replicates.

    """
    # Extract the normalized counts and log1p transform them with a pseudocount of 1
    #dds.layers['log1p'] = np.log1p(dds.layers['normed_counts'])
    
    # Extract the normalized counts of the genes of interest
    dds_goi = dds[:, dds.var_names.isin(gene_list)]

    # Create a DataFrame for plotting the heatmap
    goi_norm_counts_df = pd.DataFrame(
        dds_goi.layers['normed_counts'].T, 
        index=dds_goi.var_names, 
        columns=dds_goi.obs_names
    )

    # Extract the oxidative damage response genes from the WP_OXIDATIVE_DAMAGE_RESPONSE gene set
    oxidative_damage_genes = gp.parser.read_gmt('../WP_OXIDATIVE_DAMAGE_RESPONSE.gmt')["WP_OXIDATIVE_DAMAGE_RESPONSE"]

    # Subset the AnnData object to include only the oxidative damage response genes
    oxidative_damage_response_genes_dds = dds[:, dds.var_names.isin(oxidative_damage_genes)]

    # Create a DataFrame of the normalized counts for the oxidative damage response genes
    oxidative_damage_genes_norm_counts_df = pd.DataFrame(
        oxidative_damage_response_genes_dds.layers['normed_counts'].T, 
        index=oxidative_damage_response_genes_dds.var_names, 
        columns=oxidative_damage_response_genes_dds.obs_names
    )

    # Compute the mean normalized counts for the oxidative damage response genes
    oxidative_damage_genes_mean_norm_counts = oxidative_damage_genes_norm_counts_df.mean().tolist()

    # Add the mean normalized counts to the goi_norm_counts_df
    oxidative_damage_genes_mean_norm_counts_df=pd.DataFrame(data=[oxidative_damage_genes_mean_norm_counts], index=["WP_OXIDATIVE_\nDAMAGE_RESPONSE"], columns=oxidative_damage_response_genes_dds.obs_names)

    goi_norm_counts_df=pd.concat([goi_norm_counts_df, oxidative_damage_genes_mean_norm_counts_df])

    # Log1p transform the data
    goi_norm_counts_df = np.log1p(goi_norm_counts_df)

    # Get the indexes of control and covid conditions
    indexes_condition_control = metadata[metadata['Condition'] == "Control"].index.tolist()
    indexes_condition_covid = metadata[metadata['Condition'] == "COVID-19"].index.tolist()
    indexes_order = indexes_condition_control + indexes_condition_covid


    # Clean the data and reorder it based on gene_list and indexes_order
    cleaned_data = goi_norm_counts_df[indexes_order]
    cleaned_data = cleaned_data.reindex(gene_list+["WP_OXIDATIVE_\nDAMAGE_RESPONSE"])
    cleaned_data.fillna(0, inplace=True)
    cleaned_data = cleaned_data[sorted(cleaned_data.columns)]

    
    # Create a DataFrame to use as col_colors
    col_colors_df = pd.DataFrame(index=cleaned_data.columns)

    # highlight the control and the covid samples with different colors
    col_colors_df.loc[indexes_condition_control, 'color'] = 'blue'  # Color for control columns
    col_colors_df.loc[indexes_condition_covid, 'color'] = 'red'  # Color for covid columns


    # Generate heatmap with marked columns
    heatmap = sns.clustermap(
        cleaned_data, 
        method='average', 
        z_score=-1, 
        cmap='RdYlBu_r', 
        col_cluster=False, 
        row_cluster=False, 
        col_colors=[col_colors_df['color']],  
        figsize=(20, 9),  # Reduce figure size for compactness
        cbar_pos=(.93, .16, 0.03, 0.18), # Adjsut the colorbar position 
        dendrogram_ratio=(0.1, 0.05),  # Adjust to reduce cluster lengths
        tree_kws={"linewidths": 1},  # Increase dendrogram line thickness for visibility
    )    
    
    heatmap.ax_heatmap.set_xticklabels(heatmap.ax_heatmap.get_xticklabels(), fontsize=9)
    heatmap.ax_heatmap.set_yticklabels(heatmap.ax_heatmap.get_yticklabels(), fontsize=10)
    heatmap.ax_heatmap.set_ylabel("")



    # Create the legend 
    legend = [Patch(color=color, label=label) for label, color in {'Control': 'blue', 'COVID-19': 'red'}.items()]

    # Add legend to the plot
    plt.legend(handles=legend, loc='upper left', bbox_to_anchor=(-.1, -0.2), frameon=False)
        
    # Save the plot
    plt.savefig('../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_normalized_expression_clustermap.png', dpi=1000)
    print("The heatmap has been saved at '../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_normalized_expression_clustermap.png'")
    
    # Show the plot
    plt.show()
    return cleaned_data

# Call the function to plot the heatmap
plot_normalized_expression_clustermap(dds, gene_list, metadata)


At last we plot a heatmap depicting the log2Fold changes for the genes of interest 

In [ ]:

# Extract the Log2FoldChanges for the genes of interest
log2_fold_changes = [dea_results[dea_results.index == gene]["log2FoldChange"].values[0] for gene in gene_list]
log2_fold_changes_df = pd.DataFrame(data=log2_fold_changes, index=gene_list, columns=["log2FoldChange"])

# Set figure size (increase width to prevent y-tick clipping)
plt.figure(figsize=(.75, 11))

# Create the heatmap  
heatmap = sns.heatmap(log2_fold_changes_df, cmap='coolwarm', annot=True, fmt=".2f", cbar=False, annot_kws={"size": 10})

# Turn off x-axis ticks
plt.xticks([])  

# Ensure y-ticks are properly displayed
heatmap.yaxis.set_tick_params(labelsize=10)
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0)  # Ensure labels are readable

# Save the plot with bbox_inches='tight' to prevent label cutoff
plt.savefig('../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_log2foldchange_heatmap.png', dpi=1000)
print("The heatmap has been saved at '../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_log2foldchange_heatmap.png'")

# Show the plot
plt.show()


In [ ]:
# Extract the baseMeans for the genes of interest
goi_basemeans = [dea_results[dea_results.index == gene]["baseMean"].values[0] for gene in gene_list]

# Calculate the mean baseMean for the oxidative damage response genes
oxidative_damage_response_genes_basemeans_mean = np.mean([
    dea_results[dea_results.index == gene]["baseMean"].values[0] 
    for gene in gp.parser.read_gmt('../WP_OXIDATIVE_DAMAGE_RESPONSE.gmt')["WP_OXIDATIVE_DAMAGE_RESPONSE"]
])

# Append the mean baseMean for the oxidative damage response genes to the DataFrame
goi_basemeans_df = pd.DataFrame(
    data=goi_basemeans + [oxidative_damage_response_genes_basemeans_mean], 
    index=gene_list + ["WP_OXIDATIVE_\nDAMAGE_RESPONSE"], 
    columns=["baseMean"]
)

# Set figure size (increase width for better label visibility)
plt.figure(figsize=(0.75, 12.5))

# Create the heatmap  
heatmap = sns.heatmap(
    goi_basemeans_df, cmap='coolwarm', annot=True, fmt=".2f", cbar=False, annot_kws={"size": 10}
)

# Turn off x-axis ticks
plt.xticks([])  

# Ensure y-ticks are properly displayed
heatmap.yaxis.set_tick_params(labelsize=10)
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0)  # Ensure labels are readable

# Save the plot with bbox_inches='tight' to prevent label cutoff
plt.savefig('../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_basemean_heatmap.png', dpi=1000)
print("The heatmap has been saved at '../Results/bulk_transcriptomic_analysis/bulk_transcriptomic_analysis_basemean_heatmap.png'")

# Show the plot
plt.show()

Save the processed count matrix and metadata

In [ ]:
# Save the count matrix
counts_df.to_csv('../bulk_transcriptomic_analysis_processed_count_matrix.tsv', sep='\t')

